<a href="https://colab.research.google.com/github/hchgssarwyh/ai_fiction/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Char-based text generation with LSTM

In [ ]:
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

In [ ]:
TRAIN_TEXT_FILE_PATH = 'train_text.txt'

with open(TRAIN_TEXT_FILE_PATH) as text_file:
    text_sample = text_file.readlines()
text_sample = ' '.join(text_sample)

def text_to_seq(text_sample):
    char_counts = Counter(text_sample)
    char_counts = sorted(char_counts.items(), key = lambda x: x[1], reverse=True)

    sorted_chars = [char for char, _ in char_counts]
    print(sorted_chars)
    char_to_idx = {char: index for index, char in enumerate(sorted_chars)}
    idx_to_char = {v: k for k, v in char_to_idx.items()}
    sequence = np.array([char_to_idx[char] for char in text_sample])
    
    return sequence, char_to_idx, idx_to_char

sequence, char_to_idx, idx_to_char = text_to_seq(text_sample)

[' ', '\n', 'о', 'а', 'е', 'т', 'и', 'н', 'л', 'с', 'р', 'в', 'к', 'у', 'д', 'м', 'п', 'ь', 'я', 'ы', '.', 'й', 'г', 'б', 'з', ',', 'ч', 'ж', 'х', 'ш', 'ю', '!', 'П', 'С', 'В', '…', 'Н', '—', 'ц', 'К', '?', 'О', 'Т', 'М', 'Д', 'Г', 'И', 'З', 'Б', 'Л', 'А', 'щ', '-', ':', '»', '«', 'У', 'Р', 'Ч', 'Х', 'э', 'Я', 'Ж', 'ф', 'Е', '*', 'Ш', 'Ф', 'Э', 'ё', '–', 'Ц', 'e', '0', 'o', 'ъ', 't', 'a', '(', 'n', 'k', ')', 'C', 'X', 'r', '"', 'Й', 'c', 'Щ', "'", 'p', 'm', 'Ь', 's', 'i', '9', '1', 'w', 'B', 'M', 'd', '2', 'Ю', 'N', 'b', 'u', 'D', 'l', 'K', 'Ё', 'V', 'R', 'h', 'U', 'I', '<', '8', '4', '#', 'H', '6', '7']


In [ ]:
SEQ_LEN = 256
BATCH_SIZE = 16

def get_batch(sequence):
    trains = []
    targets = []
    for _ in range(BATCH_SIZE):
        batch_start = np.random.randint(0, len(sequence) - SEQ_LEN)
        chunk = sequence[batch_start: batch_start + SEQ_LEN]
        train = torch.LongTensor(chunk[:-1]).view(-1, 1)
        target = torch.LongTensor(chunk[1:]).view(-1, 1)
        trains.append(train)
        targets.append(target)
    return torch.stack(trains, dim=0), torch.stack(targets, dim=0)

In [ ]:
def evaluate(model, char_to_idx, idx_to_char, start_text=' ', prediction_len=200, temp=0.3):
    hidden = model.init_hidden()
    idx_input = [char_to_idx[char] for char in start_text]
    train = torch.LongTensor(idx_input).view(-1, 1, 1).to(device)
    predicted_text = start_text
    
    _, hidden = model(train, hidden)
        
    inp = train[-1].view(-1, 1, 1)
    
    for i in range(prediction_len):
        output, hidden = model(inp.to(device), hidden)
        output_logits = output.cpu().data.view(-1)
        p_next = F.softmax(output_logits / temp, dim=-1).detach().cpu().data.numpy()        
        top_index = np.random.choice(len(char_to_idx), p=p_next)
        inp = torch.LongTensor([top_index]).view(-1, 1, 1).to(device)
        predicted_char = idx_to_char[top_index]
        predicted_text += predicted_char
    
    return predicted_text

In [ ]:
class TextRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size, embedding_size, n_layers=1):
        super(TextRNN, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.embedding_size = embedding_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(self.input_size, self.embedding_size)
        self.lstm = nn.LSTM(self.embedding_size, self.hidden_size, self.n_layers)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.hidden_size, self.input_size)
        
    def forward(self, x, hidden):
        x = self.encoder(x).squeeze(2)
        out, (ht1, ct1) = self.lstm(x, hidden)
        out = self.dropout(out)
        x = self.fc(out)
        return x, (ht1, ct1)
    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device),
               torch.zeros(self.n_layers, batch_size, self.hidden_size, requires_grad=True).to(device))

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = TextRNN(input_size=len(idx_to_char), hidden_size=128, embedding_size=128, n_layers=2)
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, amsgrad=True)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    patience=5, 
    verbose=True, 
    factor=0.5
)

n_epochs = 12000
loss_avg = []

for epoch in range(n_epochs):
    model.train()
    train, target = get_batch(sequence)
    train = train.permute(1, 0, 2).to(device)
    target = target.permute(1, 0, 2).to(device)
    hidden = model.init_hidden(BATCH_SIZE)

    output, hidden = model(train, hidden)
    loss = criterion(output.permute(1, 2, 0), target.squeeze(-1).permute(1, 0))
    
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    loss_avg.append(loss.item())
    if len(loss_avg) >= 50:
        mean_loss = np.mean(loss_avg)
        print(f'Loss: {mean_loss}')
        scheduler.step(mean_loss)
        loss_avg = []
        model.eval()
        predicted_text = evaluate(model, char_to_idx, idx_to_char)
       # print(predicted_text)

Loss: 2.944972162246704
Loss: 2.4089386653900147
Loss: 2.2373326921463015
Loss: 2.1201955461502076
Loss: 2.03681024312973
Loss: 1.95648743391037
Loss: 1.8821537971496582
Loss: 1.8260656023025512
Loss: 1.7737565517425538
Loss: 1.7200907063484192
Loss: 1.6762205815315248
Loss: 1.6482713985443116
Loss: 1.599206771850586
Loss: 1.5734911823272706
Loss: 1.5496368741989135
Loss: 1.5120799112319947
Loss: 1.4862719368934632
Loss: 1.4616685485839844
Loss: 1.4395147156715393
Loss: 1.4229584002494813
Loss: 1.402214617729187
Loss: 1.3805783343315126
Loss: 1.3670485258102416
Loss: 1.3429681253433228
Loss: 1.322350583076477
Loss: 1.3170025777816772
Loss: 1.2920494699478149
Loss: 1.293750114440918
Loss: 1.278380992412567
Loss: 1.2762347912788392
Loss: 1.2575621700286865
Loss: 1.2510270404815673
Loss: 1.2474600863456726
Loss: 1.2373412466049194
Loss: 1.221976134777069
Loss: 1.1957749271392821
Loss: 1.2147232460975648
Loss: 1.1863037896156312
Loss: 1.1874105834960937
Loss: 1.1902997851371766
Loss: 1.168

In [ ]:
import telebot;
from telebot import types
bot = telebot.TeleBot('5912676709:AAFogJDypnO9nA9GZ9bKMB4Kw4ul_d7CYQk');

word = '';

@bot.message_handler(content_types=['text'])
def get_first_messages(message):
    if message.text == "Напиши стих":
        bot.send_message(message.from_user.id, "Введи первое слово")
        bot.register_next_step_handler(message, get_word);
    elif message.text == "/help":
        bot.send_message(message.from_user.id, 'Спроси "Напиши стих"')
    else:
        bot.send_message(message.from_user.id, "Я тебя не понимаю. Напиши /help.")

def get_word(message): #получаем слово
    global word;
    word = message.text;

    model.eval()

    bot.send_message(message.from_user.id, 
    evaluate(
    model, 
    char_to_idx, 
    idx_to_char, 
    temp=0.3, 
    prediction_len=200, 
    start_text = word + ' '
    ))

    keyboard = types.InlineKeyboardMarkup(); #наша клавиатура
    key_yes = types.InlineKeyboardButton(text='Да', callback_data='yes'); #кнопка «Да»
    keyboard.add(key_yes); #добавляем кнопку в клавиатуру
    key_no= types.InlineKeyboardButton(text='Нет', callback_data='no');
    keyboard.add(key_no);
    question = 'Попробовать еще?';
    bot.send_message(message.from_user.id, text=question, reply_markup=keyboard)

@bot.callback_query_handler(func=lambda call: True)
def callback_worker(call):
    if call.data == "yes": #call.data это callback_data, которую мы указали при объявлении кнопки
        bot.send_message(call.message.chat.id, 'Введи первое слово');
     #   message = message.text
        bot.register_next_step_handler(call.message, get_word);
    elif call.data == "no":
        bot.send_message(call.message.chat.id, ':(');

In [ ]:
bot.polling(none_stop=True, interval=0) 